In [1]:
import pandas as pd

import plotly.express as px
import numpy as np


import os


from tqdm.notebook import tqdm
tqdm.pandas()
pd.options.display.max_columns = 999



import plotly.graph_objects as go
import ipyvuetify as v

from ipywidgets import interact, Checkbox, Select, Output, widgets
import ipywidgets
from datetime import datetime

from IPython.display import display

In [2]:
flights_df=pd.read_csv('../03_routes_schedule/data/final_26_09.csv',  keep_default_na=False, na_values=['','NaN'], index_col=0).dropna(subset=['departure_lon','arrival_lon'])
flights_df['acft_icao']=flights_df['acft_icao'].fillna('Unknown Aircraft')
flights_df['airline_iata']=flights_df['airline_iata'].fillna('Unknown Airline')
flights_df['CO2 (Mt)']=flights_df['co2']/1e9
flights_df['ASK (Bn)']=flights_df['ask']/1e9
flights_df['Seats (Mn)']=flights_df['seats']/1e6

### Aestethics
continent_codes = {
    'AF': 'Africa',
    'AS': 'Asia',
    'EU': 'Europe',
    'NA': 'North America',
    'SA': 'South America',
    'OC': 'Oceania',
    'AN': 'Antarctica'
}

color_discrete_map={'AS':'#EE9B00',
                    'AF':'#E9D8A6',
                    'EU':'#005F73',
                    'NA':'#9B2226',
                    'OC':'#94D2BD',
                    'SA':'#BB3E03',
                   '(?)':'lightgrey'}


flights_df['departure_continent_name'] = flights_df['departure_continent'].map(continent_codes)
flights_df['arrival_continent_name'] = flights_df['arrival_continent'].map(continent_codes)



country_codes=pd.read_csv('../03_routes_schedule/data/country_codes.csv',keep_default_na=False, na_values=['','NaN'], index_col=0)

flights_df=flights_df.merge(country_codes[['Country_Name','Two_Letter_Country_Code','Three_Letter_Country_Code']], left_on='departure_country', 
                 right_on='Two_Letter_Country_Code', how='left').rename(columns={'Country_Name':'departure_country_name','Three_Letter_Country_Code':'departure_ISO3'}).drop(columns={'Two_Letter_Country_Code'})
flights_df=flights_df.merge(country_codes[['Country_Name','Two_Letter_Country_Code','Three_Letter_Country_Code']], left_on='arrival_country', 
                 right_on='Two_Letter_Country_Code', how='left').rename(columns={'Country_Name':'arrival_country_name','Three_Letter_Country_Code':'arrival_ISO3'}).drop(columns={'Two_Letter_Country_Code'})

# adding a bilateral continent columns (e.g. AF-> EU ==> AF-EU; EU-> AF ==> AF-EU)
flights_df_conti=flights_df.copy()
flights_df_conti.dropna(subset={'departure_continent','arrival_continent'}, inplace=True)
flights_df_conti['group_col'] = flights_df_conti[['departure_continent', 'arrival_continent']].apply(lambda x: tuple(sorted(x)), axis=1)



# Groupping flights keeping the directions
continental_flows=flights_df_conti.groupby(['departure_continent','departure_continent_name','arrival_continent','arrival_continent_name'])[['seats','ask','fuel_burn','co2','CO2 (Mt)', 'ASK (Bn)', 'Seats (Mn)']].sum().reset_index()

# Groupping flights without the directions
continental_flows_non_dir=flights_df_conti.groupby(['group_col'])[['seats','ask','fuel_burn','co2','CO2 (Mt)', 'ASK (Bn)', 'Seats (Mn)']].sum().reset_index()




## Adding continent coordinates for plots

conti_data={'AF': {'lat': 11.5024338, 'lon': 17.7578122},
            'AS': {'lat': 51.2086975, 'lon': 89.2343748},
            'EU': {'lat': 51.0, 'lon': 10.0},
            'NA': {'lat': 51.0000002, 'lon': -109.0},
            'SA': {'lat': -21.0002179, 'lon': -61.0006565},
            'OC': {'lat': -12.7725835, 'lon': 173.7741688},
            'AN': {'lat': -79.4063075, 'lon': 0.3149312}}


        
continental_flows['dep_lon']=continental_flows.apply(lambda row: conti_data[row.departure_continent]['lon'], axis=1)
continental_flows['dep_lat']=continental_flows.apply(lambda row: conti_data[row.departure_continent]['lat'], axis=1)
continental_flows['arr_lon']=continental_flows.apply(lambda row: conti_data[row.arrival_continent]['lon']-10, axis=1)
continental_flows['arr_lat']=continental_flows.apply(lambda row: conti_data[row.arrival_continent]['lat'], axis=1)

continental_flows_non_dir['dep_lon']=continental_flows_non_dir.apply(lambda row: conti_data[row.group_col[0]]['lon'], axis=1)
continental_flows_non_dir['dep_lat']=continental_flows_non_dir.apply(lambda row: conti_data[row.group_col[0]]['lat'], axis=1)
continental_flows_non_dir['arr_lon']=continental_flows_non_dir.apply(lambda row: conti_data[row.group_col[1]]['lon'], axis=1)
continental_flows_non_dir['arr_lat']=continental_flows_non_dir.apply(lambda row: conti_data[row.group_col[1]]['lat'], axis=1)



# Creating a nother df to compute the flights remainign in and going out of a continent for plotting

sub=continental_flows.copy()
sub['inside'] = (sub['departure_continent'] == sub['arrival_continent']) 
conti_scatter=sub.groupby(['departure_continent','departure_continent_name','inside','dep_lon','dep_lat'])[['seats','ask','fuel_burn', 'co2', 'CO2 (Mt)', 'ASK (Bn)', 'Seats (Mn)']].sum().reset_index()


## Countries df
country_flows=flights_df.groupby(['departure_country','arrival_country','departure_country_name','arrival_country_name','departure_ISO3','arrival_ISO3'])[['seats','ask','fuel_burn','co2','CO2 (Mt)','ASK (Bn)','Seats (Mn)']].sum().reset_index()
country_fixed=flights_df.groupby(['departure_country','departure_country_name','departure_ISO3'])[['seats','ask','fuel_burn','co2','CO2 (Mt)','ASK (Bn)','Seats (Mn)']].sum().reset_index()

In [3]:
### DEFINE PLOTS --> Migration in backend todo 


def continental_treemap_plot(continental_flows, value_watched_conti):
    fig = px.treemap(continental_flows,
                     path=[px.Constant("World"),'departure_continent_name', 'arrival_continent_name'],
                     values=value_watched_conti,
                     color='arrival_continent',
                     color_discrete_map=color_discrete_map,
                     title='Treemap for {}'.format(value_watched_conti)
                    )
    
    fig.update_layout(margin=dict(l=5, r=5, t=60, b=5))
    fig.update_traces(
        marker=dict(cornerradius=5),
    )

    if value_watched_conti=='CO2 (Mt)':
        fig.update_traces(hovertemplate='Flow=%{id}<br>CO<sub>2</sub>=%{value:.2f} (Mt)')
    elif value_watched_conti=='ASK (Bn)':
        fig.update_traces(hovertemplate='Flow=%{id}<br>ASK=%{value:.2f} (Bn)')
    elif value_watched_conti=='Seats (Mn)':
        fig.update_traces(hovertemplate='Flow=%{id}<br>Seats=%{value:.2f} (Mn)')
        
    return fig

def distance_histogramm_plot_continent(flights_df_conti, value_watched_conti):

    fig = px.histogram(
        flights_df_conti,
        x="distance_km",
        y=value_watched_conti,
        color="arrival_continent",
        color_discrete_map=color_discrete_map,
        title='Repartition of {} by flight distance'.format(value_watched_conti),
        histfunc="sum",
    )

    fig.update_traces(xbins=dict( 
        start=0.0,
        end=flights_df_conti.distance_km.max(),
        size=500))
    
    fig.update_layout(
        # title="Histogram of CO2 Emissions by Distance and Arrival Continent",
        xaxis_title="Distance (km)",
        yaxis_title=value_watched_conti,
        legend_title="Arrival Continent",
        showlegend=True,
    )
    fig.update_layout(
        margin=dict(l=5, r=5, t=60, b=5),
        legend=dict(
            yanchor="top",
            xanchor="right",
            x=0.9,
            bgcolor='rgba(220, 220, 220, 0.7)'
        )
    )

    fig.update_layout()
    if value_watched_conti=='CO2 (Mt)':
        fig.update_traces(hovertemplate='Arrival Continent=%{customdata}<br>Distance group (km)=%{x}<br>CO2 (Mt)=%{y:.2f}<extra></extra>', customdata=flights_df_conti['arrival_continent_name'])
    elif value_watched_conti=='ASK (Bn)':
        fig.update_traces(hovertemplate='Arrival Continent=%{customdata}<br>Distance group (km)=%{x}<br>ASK (Bn)=%{y:.2f}<extra></extra>', customdata=flights_df_conti['arrival_continent_name'])
    elif value_watched_conti=='Seats (Mn)':
        fig.update_traces(hovertemplate='Arrival Continent=%{customdata}<br>Distance group (km)=%{x}<br>Seats (Mn)=%{y:.2f}<extra></extra>', customdata=flights_df_conti['arrival_continent_name'])

    
    return fig


def continental_map_plot(conti_scatter, continental_flows_non_dir, value_watched_conti):
    # Create the scattergeo figure
    fig = go.Figure()


    for i in range(len(continental_flows_non_dir)):
        fig.add_trace(
            go.Scattergeo(
                lon = [continental_flows_non_dir['dep_lon'][i], continental_flows_non_dir['arr_lon'][i]],
                lat = [continental_flows_non_dir['dep_lat'][i], continental_flows_non_dir['arr_lat'][i]],
                mode = 'lines',
                line = dict(width = continental_flows_non_dir[value_watched_conti][i]/(.05*max(continental_flows_non_dir[value_watched_conti])), color='#EE9B00'),
                opacity=1,
                showlegend=False
            )
        )


    conti_scatter.sort_values(by=value_watched_conti, ascending=False, inplace=True)
    # Define the color mapping
    color_map = {True:'#005F73', False:'#EE9B00'}

    fig.add_trace(go.Scattergeo(
        lon = conti_scatter['dep_lon'],
        lat = conti_scatter['dep_lat'],
        text = conti_scatter[value_watched_conti],
        mode = 'markers',
        marker = dict(
            size = conti_scatter[value_watched_conti]/(.01*max(continental_flows_non_dir[value_watched_conti])),
            color=[color_map[val] for val in conti_scatter['inside']],
            line=dict(width=0.5, color='white'), opacity=1,
            # name='yy'
        ),
        customdata=conti_scatter[['departure_continent_name','inside']],
        hovertemplate = "Departure Continent: %{customdata[0]}<br>" +
                "Flights inside zone: %{customdata[1]}<br>" +
                value_watched_conti+": %{text:.1f}<br>" +
                "<extra></extra>",
        showlegend=False
    ))


    fig.add_trace(go.Scattergeo(
        lon=[None],
        lat=[None],
        mode='markers',
        marker=dict(size=50, color='#005F73'),  # Define color and size for 'Flights remaining in the zone'
        name='Flights remaining in the zone',  # Legend label for this category
    ))

    fig.add_trace(go.Scattergeo(
        lon=[None],
        lat=[None],
        mode='markers',
        marker=dict(size=50, color='#EE9B00'),  # Define color and size for 'Flights going out of the zone'
        name='Flights going out of the zone',  # Legend label for this category
    ))

    fig.update_layout(showlegend=True, height=800, title='Continental flows of {}'.format(value_watched_conti))
    fig.update_layout(margin=dict(l=5, r=5, t=60, b=5), 
                      legend=dict(
                        yanchor="top",
                        y=0.95,
                        xanchor="right",
                        x=0.9,
                        bgcolor='rgba(220, 220, 220, 0.7)'
                      )
                     )  # Adjust layout margins and padding
    return fig


In [4]:
def countries_map_plot(country_flows, value_watched_ctry):
    # Create the scattergeo figure
    fig = go.Figure(
        
    )


    for i in range(len(country_flows)):
        fig.add_trace(
            go.Scattergeo(
                locations=[country_flows['departure_ISO3'][i],country_flows['arrival_ISO3'][i]],
                locationmode="ISO-3",
                mode = 'lines',
                line = dict(width = country_flows[value_watched_ctry][i]/(.05*max(country_flows[value_watched_ctry])), color='#EE9B00'),
                opacity=1
            )
        )
        
    fig.add_trace(go.Scattergeo(
        locations=country_flows['arrival_ISO3'],
        locationmode="ISO-3",
        hoverinfo = 'text',
        text = country_flows[value_watched_ctry],
        mode = 'markers',
        marker = dict(
            size = country_flows[value_watched_ctry]/(.01*max(country_flows[value_watched_ctry])),
            color= 'purple',
            line=dict(width=0.5, color='white'), opacity=1
        ),
        customdata=country_flows[['departure_country_name','arrival_country_name']],
        hovertemplate = "%{customdata[0]}" + " TO " + "%{customdata[1]}<br>" +
                value_watched_ctry+": %{text:.0f}<br>" +
                "<extra></extra>",
    ))
    

    fig.update_geos(showcountries=True)
    fig.update_layout(showlegend=False, height=800, title='Coutry pair flows of {}'.format(value_watched_ctry))
    fig.update_layout(margin=dict(l=5, r=5, t=60, b=5))  # Adjust layout margins and padding
    return fig



def countries_global_plot(country_fixed, value_watched_ctry):
    
    fig = go.Figure()
    fig.add_trace(go.Scattergeo(
            locations=country_fixed['departure_ISO3'],
            locationmode="ISO-3",
            hoverinfo = 'text',
            text = country_fixed[value_watched_ctry],
            mode = 'markers',
            marker = dict(
                size = country_fixed[value_watched_ctry]/(.005*max(country_fixed[value_watched_ctry])),
                color= '#EE9B00',
                line=dict(width=0.5, color='white'), opacity=1
            ),
            customdata=country_fixed[['departure_country_name']],
            hovertemplate = "%{customdata[0]}<br>" +
                    value_watched_ctry+": %{text:.0f}<br>" +
                    "<extra></extra>",

        ))
    fig.update_geos(showcountries=True)
    fig.update_layout(showlegend=False, height=800, title='Coutry values for {}'.format(value_watched_ctry))
    fig.update_layout(margin=dict(l=5, r=5, t=60, b=5))  # Adjust layout margins and padding
    return fig

def countries_treemap_plot(country_flows, value_watched_ctry):
    fig=px.treemap(country_flows, 
                       path=[px.Constant("Total currently selected"),'departure_country_name','arrival_country_name'], 
                       values=value_watched_ctry,
                       color_discrete_sequence=px.colors.qualitative.T10,
                       color='arrival_country_name',
                       title='Treemap for {}'.format(value_watched_ctry),
                  )
              
    fig.update_layout(margin=dict(l=5, r=5, t=60, b=5))
    fig.update_traces(
        marker=dict(cornerradius=5),
    )


    if value_watched_ctry=='co2':
        fig.update_traces(hovertemplate='Flow=%{id}<br>CO<sub>2</sub>=%{value:.2f} (lg)')
    elif value_watched_ctry=='ask':
        fig.update_traces(hovertemplate='Flow=%{id}<br>ASK=%{value:.2f}')
    elif value_watched_ctry=='seats':
        fig.update_traces(hovertemplate='Flow=%{id}<br>Seats=%{value:.2f}')
    
    return fig



def distance_histogramm_plot_country(flights_df, value_watched_ctry):
    fig = px.histogram(
        flights_df,
        x="distance_km",
        y=value_watched_ctry,
        histfunc="sum",
        title='Repartition of {} by flight distance'.format(value_watched_ctry),

    )

    fig.update_traces(xbins=dict( 
        start=0.0,
        end=flights_df.distance_km.max(),
        size=500))
    
    fig.update_layout(
        # title="Histogram of CO2 Emissions by Distance and Arrival Continent",
        xaxis_title="Distance (km)",
        yaxis_title=value_watched_ctry,
        showlegend=False,
    )

    fig.update_layout(
        margin=dict(l=5, r=5, t=60, b=5),
    )

    if value_watched_ctry=='co2':
        fig.update_traces(hovertemplate='Distance group (km)=%{x}<br>CO2 (kg)=%{y:.0f}<extra></extra>')
    elif value_watched_ctry=='ask':
        fig.update_traces(hovertemplate='Distance group (km)=%{x}<br>ASK=%{y:.0f}<extra></extra>')
    elif value_watched_ctry=='seats':
        fig.update_traces(hovertemplate='Distance group (km)=%{x}<br>Seats=%{y:.0f}<extra></extra>')
    return fig


    
def aircraft_pie(flights_df, value_watched_ctry):
    top_aircraft = flights_df.groupby('acft_icao')[value_watched_ctry].sum().nlargest(10)
    other_total = flights_df[value_watched_ctry].sum() - top_aircraft.sum()
    top_aircraft.loc['Other'] = other_total
    fig = px.pie(
        values=top_aircraft,
        names=top_aircraft.index,
        labels={'names': 'Aircraft', 'values': value_watched_ctry},
    )
    fig.update_traces(textposition='inside')
    fig.update_layout(
        margin=dict(l=60, r=60, t=60, b=60),
        title="{} by aircraft model".format(value_watched_ctry),
        legend=dict(
            title='Aircraft type:',
        )
    )
    return fig
    


    
def aircraft_user_pie(flights_df, value_watched_ctry):
    top_airlines = flights_df.groupby('airline_iata')[value_watched_ctry].sum().nlargest(10)
    other_total = flights_df[value_watched_ctry].sum() - top_airlines.sum()
    top_airlines.loc['Other'] = other_total
    fig = px.pie(
        values=top_airlines,
        names=top_airlines.index,
        labels={'names': 'Airline', 'values': value_watched_ctry},
    )
    fig.update_traces(textposition='inside')
    fig.update_layout(
        margin=dict(l=60, r=60, t=60, b=60),
        title="{} by aircraft airline (code)".format(value_watched_ctry),
        legend=dict(
            title='Airline IATA code:',
        )
    )
    return fig
    

In [5]:
def flights_map_plot(flights_gpb_df, value_watched_flights):
    # Create the scattergeo figure
    fig = go.Figure()

    
    maxwidth=.05*max(flights_gpb_df[value_watched_flights])

    for i in range(len(flights_gpb_df)):
        fig.add_trace(
            go.Scattergeo(
                lon = [flights_gpb_df['departure_lon'][i], flights_gpb_df['arrival_lon'][i]],
                lat = [flights_gpb_df['departure_lat'][i], flights_gpb_df['arrival_lat'][i]],
                mode = 'lines',
                line = dict(width = flights_gpb_df[value_watched_flights][i]/maxwidth, color='#EE9B00'),
                opacity=0.6
            )
        )
        
    #group by airport
    
    airport_df=flights_gpb_df.groupby('iata_arrival').agg({
            'co2': 'sum',
            'ask': 'sum',
            'seats': 'sum',
            'arrival_lon':'first',
            'arrival_lat':'first'}).reset_index()
        
    fig.add_trace(go.Scattergeo(
        lon=airport_df['arrival_lon'],
        lat=airport_df['arrival_lat'],
        hoverinfo = 'text',
        text = airport_df[value_watched_flights],
        mode = 'markers',
        marker = dict(
            size = airport_df[value_watched_flights]/(.01*max(airport_df[value_watched_flights])),
            opacity=0.8,
            line=dict(width=0.5, color='white'), 
        ),
        customdata=airport_df['iata_arrival'],
        hovertemplate = "Flights to " + "%{customdata}<br>" +
            value_watched_flights+": %{text:.0f}<br>" +
            "<extra></extra>",

    ))
    

    fig.update_geos(showcountries=True)
    fig.update_layout(showlegend=False, height=800, title='Route values for {}'.format(value_watched_flights))
    fig.update_layout(margin=dict(l=5, r=5, t=60, b=5))  # Adjust layout margins and padding
    return fig



def flights_treemap_plot(flights_df, value_watched_flights):
    fig=px.treemap(flights_df, 
               path=[px.Constant("Total currently selected"),'iata_departure','iata_arrival','airline_iata','acft_icao'], 
               values=value_watched_flights,
                color_discrete_sequence=px.colors.qualitative.T10,
                title='Treemap for {}'.format(value_watched_flights))
              
    fig.update_layout(margin=dict(l=5, r=5, t=60, b=5))
    fig.update_traces(
        marker=dict(cornerradius=5),
    )

    if value_watched_flights=='co2':
        fig.update_traces(hovertemplate='Flow=%{id}<br>CO<sub>2</sub>=%{value:.2f} (lg)')
    elif value_watched_flights=='ask':
        fig.update_traces(hovertemplate='Flow=%{id}<br>ASK=%{value:.2f}')
    elif value_watched_flights=='seats':
        fig.update_traces(hovertemplate='Flow=%{id}<br>Seats=%{value:.2f}')
    
    return fig


def distance_histogramm_plot_flights(flights_df, value_watched_flights):
    fig = px.histogram(
        flights_df,
        x="distance_km",
        y=value_watched_flights,
        histfunc="sum",
        title='Repartition of {} by flight distance'.format(value_watched_flights),

    )
    
    fig.update_layout(
        # title="Histogram of CO2 Emissions by Distance and Arrival Continent",
        xaxis_title="Distance (km)",
        yaxis_title=value_watched_flights,
        showlegend=True,
    )

    fig.update_traces(xbins=dict( 
        start=0.0,
        end=flights_df.distance_km.max(),
        size=500))

    fig.update_layout(
        margin=dict(l=5, r=5, t=60, b=5),
    )

    if value_watched_flights=='co2':
        fig.update_traces(hovertemplate='Distance group (km)=%{x}<br>CO2 (kg)=%{y:.0f}<extra></extra>')
    elif value_watched_flights=='ask':
        fig.update_traces(hovertemplate='Distance group (km)=%{x}<br>ASK=%{y:.0f}<extra></extra>')
    elif value_watched_flights=='seats':
        fig.update_traces(hovertemplate='Distance group (km)=%{x}<br>Seats=%{y:.0f}<extra></extra>')
    return fig



    
def aircraft_pie_flights(flights_df, value_watched_flights):
    top_aircraft = flights_df.groupby('acft_icao')[value_watched_flights].sum().nlargest(10)
    other_total = flights_df[value_watched_flights].sum() - top_aircraft.sum()
    top_aircraft.loc['Other'] = other_total
    fig = px.pie(
        values=top_aircraft,
        names=top_aircraft.index,
        labels={'names': 'Aircraft', 'values': value_watched_flights},
    )
    fig.update_traces(textposition='inside')
    fig.update_layout(
        margin=dict(l=60, r=60, t=60, b=60),
        title="{} by aircraft model".format(value_watched_flights),
        legend=dict(
            title='Aircraft type:',
        )
    )
    return fig

    


    
def aircraft_user_pie_flights(flights_df, value_watched_flights):
    top_airlines = flights_df.groupby('airline_iata')[value_watched_flights].sum().nlargest(10)
    other_total = flights_df[value_watched_flights].sum() - top_airlines.sum()
    top_airlines.loc['Other'] = other_total
    fig = px.pie(
        values=top_airlines,
        names=top_airlines.index,
        labels={'names': 'Airline', 'values': value_watched_flights},
    )
    fig.update_traces(textposition='inside')
    fig.update_layout(
        margin=dict(l=60, r=60, t=60, b=60),
        title="{} by aircraft model".format(value_watched_flights),
        legend=dict(
            title='Aircraft type:',
        )
    )
    return fig


In [6]:
### MAIN FRONT



### CONTINENTAL FLOWS FRONTEND
continent_select = v.Select(v_model=['AF', 'AS', 'NA', 'SA', 'EU', 'OC'],
                   multiple=True,
                   clearable=True,
                   chips=True,
                   items=['AF', 'AS', 'NA', 'SA', 'EU', 'OC'])



value_watched_radio = v.RadioGroup(
    v_model='CO2 (Mt)',  # Set the initial selected value here
    row=True,
    children=[
        v.Radio(label='CO\u2082', value='CO2 (Mt)'),
        v.Radio(label='ASK', value='ASK (Bn)'),
        v.Radio(label='SEATS', value='Seats (Mn)'),
    ],
     class_='mb-3'
)




output_figure_conti1 = Output()
output_figure_conti2 = Output()
output_figure_conti3 = Output()


def render_initial_plots_conti():
    
    continental_flows_non_dir[['AV1', 'AV2']] = continental_flows_non_dir['group_col'].copy().apply(lambda x: pd.Series(x))
    with output_figure_conti1:
        fig_conti_1=continental_map_plot(conti_scatter, continental_flows_non_dir,'CO2 (Mt)')
        display(fig_conti_1)
        
    with output_figure_conti2:
        fig_conti_2=continental_treemap_plot(continental_flows,'CO2 (Mt)')
        display(fig_conti_2)
        
    with output_figure_conti3:
        fig_conti_3=distance_histogramm_plot_continent(flights_df_conti,'CO2 (Mt)')
        display(fig_conti_3)
        
        
        
    


def plots_update_conti(change):

    filtered_values = continent_select.v_model
    value_watched_conti = value_watched_radio.v_model
    
    filtered_df_depart = conti_scatter[conti_scatter['departure_continent'].isin(filtered_values)].reset_index()
    filtered_df = continental_flows[continental_flows['departure_continent'].isin(filtered_values)].reset_index()
    filtered_fl_df = flights_df_conti[flights_df_conti['departure_continent'].isin(filtered_values)].reset_index()
    
    continental_flows_non_dir[['AV1', 'AV2']] = continental_flows_non_dir['group_col'].copy().apply(lambda x: pd.Series(x))
    filtered_non_dir=continental_flows_non_dir[(continental_flows_non_dir.AV1.isin(filtered_values))|(continental_flows_non_dir.AV2.isin(filtered_values))].reset_index()

    with output_figure_conti1:
        output_figure_conti1.clear_output(wait=True)
        fig_conti_1=continental_map_plot(filtered_df_depart, filtered_non_dir, value_watched_conti)
        display(fig_conti_1)
        
    with output_figure_conti2:
        output_figure_conti2.clear_output(wait=True)
        fig_conti_2=continental_treemap_plot(filtered_df, value_watched_conti)
        display(fig_conti_2)
        
    with output_figure_conti3:
        output_figure_conti3.clear_output(wait=True)
        fig_conti_3=distance_histogramm_plot_continent(filtered_fl_df, value_watched_conti)
        display(fig_conti_3)
    
    

    

# # Connect the event handler to the controls 

continent_select.observe(plots_update_conti, names='v_model')
value_watched_radio.observe(plots_update_conti, names='v_model')





### PAGE ARCHITECTURE

## Define the rows


row_selects = v.Row(
    justify='center',  # Center the components horizontally
    no_gutters=False,
    class_='mb-4',  # Add margin at the bottom
    children=[
        v.Col(
            cols='6',  # Adjust the column width as needed
            children=[
                v.Card(
                    outlined=True,
                    elevation=1,
                    children=[
                        v.CardText(
                            children=[
                                v.CardTitle(children="Select the 'departure' continents included"),
                                continent_select,
                            ]
                        ),
                    ]
                ),
            ],
        ),
        v.Col(
            cols='6',  # Adjust the column width as needed
            children=[
                v.Card(
                    outlined=True,
                    elevation=1,
                    children=[
                        v.CardText(
                            children=[
                                v.CardTitle(children="Value Watched"),
                                value_watched_radio,
                            ]
                        ),
                    ]
                ),
            ],
        ),
    ],
)


row_mega_map=v.Row(
            children=[
                v.Col(
                    xs12=True,
                    children=[
                        v.Card(
                            outlined=True,
                            elevation=3,
                            children=[
                                v.CardText(
                                    children=[output_figure_conti1,
                                    ]
                                ),
                                
                            ],
                        ),
                    ],
                    style_="max-width: 100%;"
                ),
            ],
        )


row_twoplots=v.Row(
                children=[
                    v.Col(
                        xs12=True,
                        lg6=True,
                        xl6=True,
                        children=[
                            v.Card(
                                outlined=True,
                                elevation=3,
                                children=[
                                    output_figure_conti2,
                                ],
                            ),
                        ],
                    ),
                    v.Col(
                        xs12=True,
                        lg6=True,
                        xl6=True,
                        children=[
                            v.Card(
                                outlined=True,
                                elevation=3,
                                children=[
                                    output_figure_conti3,
                                ],
                            ),
                        ],
                    ),
                ],
            )




# render_initial_plots_conti()

In [7]:
### COUNTRIES FRONTEND

countries_autocomplete = v.Autocomplete(
    v_model=['World'],
    clearable=True,
    chips=True,
    label="Autocomplete",
    items=list(np.append(country_flows.departure_country_name.unique(),'World')),
    multiple=True,
    variant="outlined"
)

def select_world(widget, event, data):
    countries_autocomplete.v_model = ['World']

select_world_button = v.Btn(children=["World View"], variant='outlined')
select_world_button.on_event("click", select_world)


value_watched_radio_ctry = v.RadioGroup(
    v_model='co2',  # Set the initial selected value here
    row=True,
    children=[
        v.Radio(label='CO\u2082 (kg)', value='co2'),
        v.Radio(label='ASK', value='ask'),
        v.Radio(label='SEATS', value='seats'),
    ],
     class_='mb-3'
)

# v-btn-toggle switch between the two main figures
toggle_button_ctry = v.BtnToggle(v_model='map', variant='outlined', children=[
    v.Btn(value='map', children=['Map'] ,variant='outlined'),
    v.Btn(value='tree', children=['Tree'], variant='outlined'),
])





output_figure_ctry1 = Output()
output_figure_ctry2 = Output()
output_figure_ctry3 = Output()
output_figure_ctry4 = Output()


def render_initial_plots_ctry():
    with output_figure_ctry1:
        fig_ctry_1=countries_global_plot(country_fixed, 'co2')
        display(fig_ctry_1)
        
    with output_figure_ctry2:
        fig_ctry_2=distance_histogramm_plot_country(flights_df, 'co2')
        display(fig_ctry_2)
        
    with output_figure_ctry3:
        fig_ctry_3=aircraft_pie(flights_df, 'co2')
        display(fig_ctry_3)
        
    with output_figure_ctry4:
        fig_ctry_4=aircraft_user_pie(flights_df, 'co2')
        display(fig_ctry_4)
        
        

def plots_update_ctry(change):
    
    filtered_values = countries_autocomplete.v_model
    value_watched_ctry = value_watched_radio_ctry.v_model
    active_main_graph_country=toggle_button_ctry.v_model
    
    if filtered_values[0]=='World':         
        with output_figure_ctry1:
            output_figure_ctry1.clear_output(wait=True)
            if active_main_graph_country=='map':
                fig_ctry_1=countries_global_plot(country_fixed, value_watched_ctry)
            else:
                fig_ctry_1=countries_treemap_plot(country_flows, value_watched_ctry)
            display(fig_ctry_1)

        with output_figure_ctry2:
            output_figure_ctry2.clear_output(wait=True)
            fig_ctry_2=distance_histogramm_plot_country(flights_df, value_watched_ctry)
            display(fig_ctry_2)

        with output_figure_ctry3:
            output_figure_ctry3.clear_output(wait=True)
            fig_ctry_3=aircraft_pie(flights_df, value_watched_ctry)
            display(fig_ctry_3)

        with output_figure_ctry4:
            output_figure_ctry4.clear_output(wait=True)
            fig_ctry_4=aircraft_user_pie(flights_df, value_watched_ctry)
            display(fig_ctry_4)   
    
    else:
        filtered_country_flows = country_flows[country_flows['departure_country_name'].isin(filtered_values)].reset_index()
        filtered_flights_df = flights_df[flights_df['departure_country_name'].isin(filtered_values)].reset_index()
        print(active_main_graph_country)
        
        with output_figure_ctry1:
            output_figure_ctry1.clear_output(wait=True)
            if active_main_graph_country=='map':
                fig_ctry_1=countries_map_plot(filtered_country_flows, value_watched_ctry)
            else:
                fig_ctry_1=countries_treemap_plot(filtered_country_flows, value_watched_ctry)
            display(fig_ctry_1)
                

        with output_figure_ctry2:
            output_figure_ctry2.clear_output(wait=True)
            fig_ctry_2=distance_histogramm_plot_country(filtered_flights_df, value_watched_ctry)
            display(fig_ctry_2)

        with output_figure_ctry3:
            output_figure_ctry3.clear_output(wait=True)
            fig_ctry_3=aircraft_pie(filtered_flights_df, value_watched_ctry)
            display(fig_ctry_3)

        with output_figure_ctry4:
            output_figure_ctry4.clear_output(wait=True)
            fig_ctry_4=aircraft_user_pie(filtered_flights_df, value_watched_ctry)
            display(fig_ctry_4)

    

    

# # Connect the event handler to the controls 

countries_autocomplete.observe(plots_update_ctry, names='v_model')
value_watched_radio_ctry.observe(plots_update_ctry, names='v_model')
toggle_button_ctry.observe(plots_update_ctry, names='v_model')





### PAGE ARCHITECTURE

## Define the rows


row_selects_country = v.Row(
    justify='center',  # Center the components horizontally
    no_gutters=False,
    class_='mb-4',  # Add margin at the bottom
    children=[
        v.Col(
            cols='6',  # Adjust the column width as needed
            children=[
                v.Card(
                    outlined=True,
                    elevation=1,
                    style_='height: 100%',
                    children=[
                        v.CardText(
                            children=[
                                v.CardTitle(children="Select the 'departure' countries included in analysis"),
                                countries_autocomplete,
                                select_world_button
                            ]
                        ),
                    ]
                ),
            ],
        ),
        v.Col(
            cols='6',  # Adjust the column width as needed
            children=[
                v.Card(
                    outlined=True,
                    elevation=1,
                    style_='height: 100%',
                    children=[
                        v.CardText(
                            children=[
                                v.CardTitle(children="Select the value watched"),
                                value_watched_radio_ctry,
                            ]
                        ),
                    ]
                ),
            ],
        ),
    ],
)


row_mega_map_country=v.Row(
            children=[
                v.Col(
                    xs12=True,
                    children=[
                        v.Card(
                            outlined=True,
                            elevation=3,
                            children=[
                                v.Html(tag="div", class_="d-flex justify-center", children=[toggle_button_ctry]),
                                v.CardText(
                                    children=[
                                        output_figure_ctry1,
                                    ]
                                ),
                                
                            ],
                        ),
                    ],
                    style_="max-width: 100%;"
                ),
            ],
        )



row_threeplots_country=v.Row(
                children=[
                    v.Col(
                        xs12=True,
                        lg4=True,
                        xl4=True,
                        children=[
                            v.Card(
                                outlined=True,
                                elevation=3,
                                children=[
                                    output_figure_ctry2,
                                ],
                            ),
                        ],
                    ),
                    v.Col(
                        xs12=True,
                        lg4=True,
                        xl4=True,
                        children=[
                            v.Card(
                                outlined=True,
                                elevation=3,
                                children=[
                                    output_figure_ctry3,
                                ],
                            ),
                        ],
                    ),
                    v.Col(
                        xs12=True,
                        lg4=True,
                        xl4=True,
                        children=[
                            v.Card(
                                outlined=True,
                                elevation=3,
                                children=[
                                    output_figure_ctry4,
                                ],
                            ),
                        ],
                    ),
                ],
            )





In [8]:
### FLIGHTS FRONTEND

departure_airport_autocomplete = v.Autocomplete(
    v_model=[],
    clearable=True,
    chips=True,
    label="Autocomplete",
    items=list(flights_df.iata_departure.unique()),
    multiple=True,
    variant="outlined"
)

def departure_select_all(widget, event, data):
    departure_airport_autocomplete.v_model = list()
    
departure_select_all_button = v.Btn(children=["Reset Filter"], color="primary", class_="ma-2")
departure_select_all_button.on_event("click", departure_select_all)


arrival_airport_autocomplete = v.Autocomplete(
    v_model=[],
    clearable=True,
    chips=True,
    label="Autocomplete",
    items=list(flights_df.iata_arrival.unique()),
    multiple=True,
    variant="outlined"
)

def arrival_select_all(widget, event, data):
    arrival_airport_autocomplete.v_model = list()

arrival_select_all_button = v.Btn(children=["Reset Filter"], color="primary", class_="ma-2")
arrival_select_all_button.on_event("click", arrival_select_all)


airline_autocomplete = v.Autocomplete(
    v_model=[],
    clearable=True,
    chips=True,
    label="Autocomplete",
    items=list(flights_df.airline_iata.unique()),
    multiple=True,
    variant="outlined"
)

def airline_select_all(widget, event, data):
    airline_autocomplete.v_model = list()

airline_select_all_button = v.Btn(children=["Reset Filter"], color="primary", class_="ma-2")
airline_select_all_button.on_event("click", airline_select_all)
    
aircraft_autocomplete = v.Autocomplete(
    v_model=[],
    clearable=True,
    chips=True,
    label="Autocomplete",
    items=list(flights_df.acft_icao.unique()),
    multiple=True,
    variant="outlined"
)
    
def aircraft_select_all(widget, event, data):
    aircraft_autocomplete.v_model = list()
    
    
aircraft_select_all_button = v.Btn(children=["Reset Filter"], color="primary", class_="ma-2")
aircraft_select_all_button.on_event("click", aircraft_select_all) 
    

value_watched_radio_flights = v.RadioGroup(
    v_model='co2',  # Set the initial selected value here
    row=True,
    children=[
        v.Radio(label='CO\u2082 (kg)', value='co2'),
        v.Radio(label='ASK', value='ask'),
        v.Radio(label='SEATS', value='seats'),
    ],
     class_='mb-3'
)


# v-btn-toggle switch between the two main figures
toggle_button_flights = v.BtnToggle(v_model='map', variant='outlined', children=[
    v.Btn(value='map', children=['Map'] ,variant='outlined'),
    v.Btn(value='tree', children=['Tree'], variant='outlined'),
])


output_figure_flights1 = Output()
output_figure_flights2 = Output()
output_figure_flights3 = Output()
output_figure_flights4 = Output()


def render_initial_plots_flights():
    with output_figure_flights1:
        print('Too much data selected for flight map rendering')
        
    with output_figure_flights2:
        fig_flights_2=distance_histogramm_plot_flights(flights_df, 'co2')
        display(fig_flights_2)
        
    with output_figure_flights3:
        fig_flights_3=aircraft_pie_flights(flights_df, 'co2')
        display(fig_flights_3)
        
    with output_figure_flights4:
        fig_flights_4=aircraft_user_pie_flights(flights_df, 'co2')
        display(fig_flights_4)
        
        

def plots_update_flights(change):
    filtered_departure = departure_airport_autocomplete.v_model
    filtered_arrival = arrival_airport_autocomplete.v_model
    filtered_airline = airline_autocomplete.v_model
    filtered_aircraft = aircraft_autocomplete.v_model
    value_watched_flights = value_watched_radio_flights.v_model
    active_main_graph_flights=toggle_button_flights.v_model
    
    filtered_flights_df=flights_df
    
    print(filtered_aircraft, filtered_airline, filtered_flights_df)
    
    
    #active departure filter
    if filtered_departure:
        if 'level_0' in filtered_flights_df.columns:
            filtered_flights_df.drop('level_0', axis=1, inplace=True)
        filtered_flights_df = filtered_flights_df[filtered_flights_df['iata_departure'].isin(filtered_departure)].reset_index()
        
    #active arrival filter
    if filtered_arrival:
        if 'level_0' in filtered_flights_df.columns:
            filtered_flights_df.drop('level_0', axis=1, inplace=True)
        filtered_flights_df = filtered_flights_df[filtered_flights_df['iata_arrival'].isin(filtered_arrival)].reset_index()
    
    #active airline filter
    if filtered_airline:
        if 'level_0' in filtered_flights_df.columns:
            filtered_flights_df.drop('level_0', axis=1, inplace=True)
        filtered_flights_df = filtered_flights_df[filtered_flights_df['airline_iata'].isin(filtered_airline)].reset_index()
    
    #active acft filter
    if filtered_aircraft:
        if 'level_0' in filtered_flights_df.columns:
            filtered_flights_df.drop('level_0', axis=1, inplace=True)
        filtered_flights_df = filtered_flights_df[filtered_flights_df['acft_icao'].isin(filtered_aircraft)].reset_index()
    
    if len(filtered_flights_df)<20000:
        
        with output_figure_flights1:
            output_figure_flights1.clear_output(wait=True)
            if active_main_graph_flights=='map':
                # # grouping flighst on and OD basis, and concatenating airline and aircraft information

                flights_df_od = filtered_flights_df.groupby(['iata_departure', 'iata_arrival']).agg({
                    'acft_icao': ', '.join,
                    'airline_iata': ', '.join,
                    'departure_lon':'first',
                    'departure_lat':'first',
                    'arrival_lon':'first',
                    'arrival_lat':'first',
                    'co2': 'sum',
                    'ask': 'sum',
                    'seats': 'sum'
                }).reset_index()


                # Function to remove duplicates from a comma-separated string
                def remove_duplicates(input_str):
                    # Split the string into a list of substrings
                    substrings = input_str.split(', ')
                    # Remove duplicates and preserve the order
                    unique_substrings = list(dict.fromkeys(substrings))
                    # Join the unique substrings back into a single string
                    result_str = ', '.join(unique_substrings)
                    return result_str

                # Apply the function to the DataFrame column
                flights_df_od['airline_iata'] = flights_df_od['airline_iata'].apply(remove_duplicates)
                flights_df_od['acft_icao'] = flights_df_od['acft_icao'].apply(remove_duplicates)
                fig_flights_1=flights_map_plot(flights_df_od, value_watched_flights)
            else:
                fig_flights_1=flights_treemap_plot(filtered_flights_df, value_watched_flights)
            
            display(fig_flights_1)
            
    else:
        with output_figure_flights1:
            output_figure_flights1.clear_output(wait=True)
            print('Too much data selected for flight map rendering')

    with output_figure_flights2:
        output_figure_flights2.clear_output(wait=True)
        fig_flights_2=distance_histogramm_plot_flights(filtered_flights_df, value_watched_flights)
        display(fig_flights_2)

    with output_figure_flights3:
        output_figure_flights3.clear_output(wait=True)
        fig_flights_3=aircraft_pie_flights(filtered_flights_df, value_watched_flights)
        display(fig_flights_3)

    with output_figure_flights4:
        output_figure_flights4.clear_output(wait=True)
        fig_flights_4=aircraft_user_pie_flights(filtered_flights_df, value_watched_flights)
        display(fig_flights_4)



    

# # Connect the event handler to the controls 

departure_airport_autocomplete.observe(plots_update_flights, names='v_model')
arrival_airport_autocomplete.observe(plots_update_flights, names='v_model')
airline_autocomplete.observe(plots_update_flights, names='v_model')
aircraft_autocomplete.observe(plots_update_flights, names='v_model')
value_watched_radio_flights.observe(plots_update_flights, names='v_model')
toggle_button_flights.observe(plots_update_flights, names='v_model')




### PAGE ARCHITECTURE

## Define the rows


row_selects_flights = v.Row(
    justify='center',  # Center the components horizontally
    no_gutters=False,
    class_='mb-4',  # Add margin at the bottom
    children=[
        v.Col(
            cols='12',  # Adjust the column width as needed
            children=[
                v.Card(
                    outlined=True,
                    elevation=1,
                    style_='width: 100%',
                    children=[
                        v.CardText(
                            children=['Caution: Accuracy could be limited in this mode. Data must therefore be used with the necessary precautions.'], 
                            class_="text-center"
                        ),
                    ]
                ),
            ],
        ),
        v.Col(
            cols='2',  # Adjust the column width as needed
            children=[
                v.Card(
                    outlined=True,
                    elevation=1,
                    style_='height: 100%',
                    children=[
                        v.CardText(
                            children=[
                                v.CardTitle(children="Select the 'departure' airport(s) included in analysis"),
                                departure_airport_autocomplete,
                                departure_select_all_button
                            ]
                        ),
                    ]
                ),
            ],
        ),
        v.Col(
            cols='2',  # Adjust the column width as needed
            children=[
                v.Card(
                    outlined=True,
                    elevation=1,
                    style_='height: 100%',
                    children=[
                        v.CardText(
                            children=[
                                v.CardTitle(children="Select the 'arrival' airport(s) included in analysis"),
                                arrival_airport_autocomplete,
                                arrival_select_all_button
                            ]
                        ),
                    ]
                ),
            ],
        ),
        v.Col(
            cols='2',  # Adjust the column width as needed
            children=[
                v.Card(
                    outlined=True,
                    elevation=1,
                    style_='height: 100%',
                    children=[
                        v.CardText(
                            children=[
                                v.CardTitle(children="Select the airline(s) included in analysis"),
                                airline_autocomplete,
                                airline_select_all_button
                            ]
                        ),
                    ]
                ),
            ],
        ),
        v.Col(
            cols='2',  # Adjust the column width as needed
            children=[
                v.Card(
                    outlined=True,
                    elevation=1,
                    style_='height: 100%',
                    children=[
                        v.CardText(
                            children=[
                                v.CardTitle(children="Select the aircraft model(s) included in analysis"),
                                aircraft_autocomplete,
                                aircraft_select_all_button
                            ]
                        ),
                    ]
                ),
            ],
        ),
        v.Col(
            cols='4',  # Adjust the column width as needed
            children=[
                v.Card(
                    outlined=True,
                    elevation=1,
                    style_='height: 100%',
                    children=[
                        v.CardText(
                            children=[
                                v.CardTitle(children="Select the value watched"),
                                value_watched_radio_flights,
                            ]
                        ),
                    ]
                ),
            ],
        ),
    ],
)


row_mega_map_flights=v.Row(
            children=[
                v.Col(
                    xs12=True,
                    children=[
                        v.Card(
                            outlined=True,
                            elevation=3,
                            children=[
                                v.Html(tag="div", class_="d-flex justify-center", children=[toggle_button_flights]),
                                v.CardText(
                                    children=[output_figure_flights1,
                                    ]
                                ),
                                
                            ],
                        ),
                    ],
                    style_="max-width: 100%;"
                ),
            ],
        )


row_threeplots_flights=v.Row(
                children=[
                    v.Col(
                        xs12=True,
                        lg4=True,
                        xl4=True,
                        children=[
                            v.Card(
                                outlined=True,
                                elevation=3,
                                children=[
                                    output_figure_flights2,
                                ],
                            ),
                        ],
                    ),
                    v.Col(
                        xs12=True,
                        lg4=True,
                        xl4=True,
                        children=[
                            v.Card(
                                outlined=True,
                                elevation=3,
                                children=[
                                    output_figure_flights3,
                                ],
                            ),
                        ],
                    ),
                    v.Col(
                        xs12=True,
                        lg4=True,
                        xl4=True,
                        children=[
                            v.Card(
                                outlined=True,
                                elevation=3,
                                children=[
                                    output_figure_flights4,
                                ],
                            ),
                        ],
                    ),
                ],
            )




In [9]:
render_initial_plots_conti()
render_initial_plots_ctry()
render_initial_plots_flights()

In [10]:
##LAYOUT


# Create the layout
v.theme.dark = False

title_layout = v.Row(
    justify="center",
    class_="mt-6 mb-0",  # Added mb-0 to remove bottom margin
    style_="background-color: white;",  
    children=[
        v.Col(
            cols="12",
            children=[
                v.Html(tag="h1", children=["AirFlows"],class_="text-center")  # Added py-4 and text-white classes
            ]
        )
    ]
)


tabs_layout= v.Tabs(_metadata={'mount_id': 'content-main'},
             fixed_tabs=True,
             background_color="indigo darken-2",
             
    children=[
        v.Tab(children=['Continental flows'], style_="color: white",),
        v.Tab(children=['Countries analysis'],style_="color: white",),
        v.Tab(children=['Flights'],style_="color: white",),
        v.TabItem(children=[v.Container(children=[row_selects, row_mega_map, row_twoplots])], style_="background-color: white;"),
        v.TabItem(children=[v.Container(children=[row_selects_country, row_mega_map_country, row_threeplots_country])], style_="background-color: white;"),
        v.TabItem(children=[v.Container(children=[row_selects_flights, row_mega_map_flights, row_threeplots_flights])], style_="background-color: white;"),
    ]
)

footer_layout = v.Footer(class_=" text-center d-flex flex-column",style_='background-color: white;', children=[
    v.Col(
        class_="text-center mt-4",
        cols="12",
        children=[
            f"{datetime.now().year} — ",
            v.Html(tag="strong", children=["©ISAE-SUPAERO"])
        ]
    )
])



layout = v.Layout(
    # class_="rounded rounded-md",
    children=[
        v.Container(children=[title_layout,
        v.Container(
            children=[
                   tabs_layout,
                footer_layout
            ])])
    ]
)


app_layout = v.App(
    id='inspire',
    style_='background-color: white;',  # Set the desired background color here
    children=[
        layout
    ]
)






In [11]:
display(app_layout)

App(children=[Layout(children=[Container(children=[Row(children=[Col(children=[Html(children=['AirFlows'], cla…